In [1]:
!pip install sagemaker

     |████████████████████████████████| 527 kB 8.1 MB/s 
     |████████████████████████████████| 49 kB 5.9 MB/s 
     |████████████████████████████████| 132 kB 60.8 MB/s 
     |████████████████████████████████| 81 kB 12.7 MB/s 
     |████████████████████████████████| 8.7 MB 45.4 MB/s 
     |████████████████████████████████| 79 kB 10.5 MB/s 
     |████████████████████████████████| 138 kB 40.6 MB/s 
     |████████████████████████████████| 65 kB 4.5 MB/s 
  Created wheel for sagemaker: filename=sagemaker-2.88.1-py2.py3-none-any.whl size=728476 sha256=94f43ec276f20df81ad5d8db675afcd6f420ced3d6361d02e32efff18ed42a13
  Stored in directory: /root/.cache/pip/wheels/24/22/dc/4532decda751afdd10730d5ba070afa0f210a676e4f173d990
  Created wheel for protobuf3-to-dict: filename=protobuf3_to_dict-0.1.5-py3-none-any.whl size=4030 sha256=2f244ed26a39070a082491f04251e051f6170fd2fde101f3c7686687be336d11
  Stored in directory: /root/.cache/pip/wheels/ce/a0/8b/995ce2fbaf0e9fe7eb91da84e99e84d1b35cfaa555f2b8f

In [80]:
import pandas as pd
import time
df = pd.read_parquet(path="/content/rfm_features.parquet")
df = df.drop(columns=["created_timestamp"])
df["event_timestamp"] = float(round(time.time()))
df["customerid"] = df['customerid'].astype(float)
df.head()

,customerid,recency,frequency,monetaryvalue,r,f,m,rfmscore,revenue6m,ltvcluster,segmenthighvalue,segmentlowValue,segmentmidvalue,event_timestamp
0,12747.0,7,35,1082.09,3,2,3,8,1666.11,1,1,0,0,1.651462e+09
1,12748.0,1,582,4336.73,3,3,3,9,18679.01,2,1,0,0,1.651462e+09
2,12749.0,8,54,782.10,3,3,3,9,2323.04,1,1,0,0,1.651462e+09
4,12823.0,63,1,459.00,0,0,2,2,765.00,1,0,1,0,1.651462e+09
7,12836.0,28,62,814.71,1,3,3,7,951.46,1,1,0,0,1.651462e+09


In [2]:
import os
os.environ["AWS_ACCESS_KEY_ID"] = "<aws_key_id>"
os.environ["AWS_SECRET_ACCESS_KEY"] = "<aws_secret_id>"
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

In [40]:
import sagemaker
import pandas as pd
from sagemaker.session import Session

prefix = 'sagemaker-featurestore-introduction'
role = "arn:aws:iam::<account_number>:role/sagemaker-iam-role"

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
s3_bucket_name = "<S3_bucket>"

In [55]:
from sagemaker.feature_store.feature_group import FeatureGroup

customers_feature_group = FeatureGroup(
    name="customer-rfm-features", sagemaker_session=sagemaker_session
)

In [56]:
customers_feature_group.load_feature_definitions(df)

[FeatureDefinition(feature_name='customerid', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='recency', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='frequency', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='monetaryvalue', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='r', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='f', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='m', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='rfmscore', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='revenue6m', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='ltvcluster', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(

In [ ]:
customers_feature_group.create(
    s3_uri=f"s3://{s3_bucket_name}/{prefix}",
    record_identifier_name="customerid",
    event_time_feature_name="event_timestamp",
    role_arn=role,
    enable_online_store=True
)

In [59]:
ingestion_manager = customers_feature_group.ingest(df.head())
ingestion_manager.wait()

In [61]:
ingestion_manager.failed_rows

[]

In [63]:
customer_id = 12747.0
sg_runtime_client = sagemaker_session.boto_session.client('sagemaker-featurestore-runtime', region_name=region)
record = sg_runtime_client.get_record(FeatureGroupName="customer-rfm-features", RecordIdentifierValueAsString=str(customer_id))
print(record)

{'ResponseMetadata': {'RequestId': '55342bbc-c69b-49ca-bbd8-2c5f9ed73565', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '55342bbc-c69b-49ca-bbd8-2c5f9ed73565', 'content-type': 'application/json', 'content-length': '729', 'date': 'Mon, 02 May 2022 01:36:27 GMT'}, 'RetryAttempts': 0}, 'Record': [{'FeatureName': 'customerid', 'ValueAsString': '12747.0'}, {'FeatureName': 'recency', 'ValueAsString': '7'}, {'FeatureName': 'frequency', 'ValueAsString': '35'}, {'FeatureName': 'monetaryvalue', 'ValueAsString': '1082.09'}, {'FeatureName': 'r', 'ValueAsString': '3'}, {'FeatureName': 'f', 'ValueAsString': '2'}, {'FeatureName': 'm', 'ValueAsString': '3'}, {'FeatureName': 'rfmscore', 'ValueAsString': '8'}, {'FeatureName': 'revenue6m', 'ValueAsString': '1666.1100000000001'}, {'FeatureName': 'ltvcluster', 'ValueAsString': '1'}, {'FeatureName': 'segmenthighvalue', 'ValueAsString': '1'}, {'FeatureName': 'segmentlowValue', 'ValueAsString': '0'}, {'FeatureName': 'segmentmidvalue', 'ValueAsSt

In [79]:
get_latest_snapshot_query = customers_feature_group.athena_query()
query = f"""SELECT *
FROM
    (SELECT *,
         row_number()
        OVER (PARTITION BY customerid
    ORDER BY  event_timestamp desc, Api_Invocation_Time DESC, write_time DESC) AS row_num
    FROM "{get_latest_snapshot_query.table_name}")
WHERE row_num = 1 and 
NOT is_deleted;"""
print(query)


SELECT *
FROM
    (SELECT *,
         row_number()
        OVER (PARTITION BY customerid
    ORDER BY  event_timestamp desc, Api_Invocation_Time DESC, write_time DESC) AS row_num
    FROM "customer-rfm-features-1651455015")
WHERE row_num = 1 and 
NOT is_deleted;


In [76]:
get_latest_snapshot_query.run(query_string=query, output_location=f"s3://{s3_bucket_name}/output")

'11501516-f345-4677-8aac-c15f79715ee5'

In [ ]:
get_latest_snapshot_query.get_query_execution()

In [78]:
latest_df = get_latest_snapshot_query.as_dataframe()
latest_df.head()

,customerid,recency,frequency,monetaryvalue,r,f,m,rfmscore,revenue6m,ltvcluster,segmenthighvalue,segmentlowvalue,segmentmidvalue,event_timestamp,write_time,api_invocation_time,is_deleted,row_num
0,12747.0,7,35,1082.09,3,2,3,8,1666.11,1,1,0,0,1.651455e+09,2022-05-02 01:36:31.398,2022-05-02 01:31:59.000,False,1
1,12823.0,63,1,459.00,0,0,2,2,765.00,1,0,1,0,1.651455e+09,2022-05-02 01:36:31.406,2022-05-02 01:32:00.000,False,1
2,12836.0,28,62,814.71,1,3,3,7,951.46,1,1,0,0,1.651455e+09,2022-05-02 01:36:31.391,2022-05-02 01:32:00.000,False,1
3,12748.0,1,582,4336.73,3,3,3,9,18679.01,2,1,0,0,1.651455e+09,2022-05-02 01:36:31.400,2022-05-02 01:31:59.000,False,1
4,12749.0,8,54,782.10,3,3,3,9,2323.04,1,1,0,0,1.651455e+09,2022-05-02 01:36:31.397,2022-05-02 01:32:00.000,False,1


In [ ]:
customers_feature_group.describe()

In [82]:
customers_feature_group.delete()

In [84]:
sagemaker_client = sagemaker_session.boto_session.client(
    "sagemaker", region_name=region
) 

In [85]:
sagemaker_client.list_feature_groups()

{'FeatureGroupSummaries': [],
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '28',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 02 May 2022 03:35:22 GMT',
   'x-amzn-requestid': '0c4167ca-83ea-462d-8059-723b80ae7941'},
  'HTTPStatusCode': 200,
  'RequestId': '0c4167ca-83ea-462d-8059-723b80ae7941',
  'RetryAttempts': 0}}